In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install opencv-python

In [1]:
import sys,os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch.utils
import cv2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from random import randint

In [3]:
mean = np.load(r"C:\Users\vigne\anaconda3\envs\Pytorch\age_gender_folder\mean.npy")
std = np.load(r"C:\Users\vigne\anaconda3\envs\Pytorch\age_gender_folder\std.npy")

In [4]:
mean=torch.tensor(mean)
std=torch.tensor(std)

In [5]:
mean.item()

0.49325109310851223

In [6]:
class myModel(nn.Module):
    def __init__(self):
        super(myModel , self).__init__()
     
        self.conv1 = nn.Conv2d(1,  32,  kernel_size=3 )
        #self.pool1  = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(32,  64, kernel_size=3 )
        self.pool2  = nn.MaxPool2d(2,2)
      
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3 )
        self.pool3  = nn.MaxPool2d(2,2)

        self.linear1 = nn.Linear(3200, 64)
        self.linear2 = nn.Linear(64,16)
        #self.linear3 = nn.Linear(64,16)
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = x.view(-1, 3200)

        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        #x = F.relu(x)
 
        return x

In [7]:
import pickle as pk
dest_path = r"C:\Users\vigne\anaconda3\envs\Pytorch\age_gender_folder\model.pkl"
model = myModel()
model = pk.load(open(dest_path,'rb'))

In [8]:
# Import face detector algorithm
face_detector = cv2.CascadeClassifier(r"C:\Users\vigne\anaconda3\envs\Pytorch\age_gender_folder\haarcascade_frontalface_default.xml")

In [9]:
    '''
# 16 classes
# MALE = 0
# 0: (0 – 2)
# 1: (4 – 6)
# 2: (8 – 12)
# 3: (15 – 20)
# 4: (25 – 32)
# 5: (38 – 43) 
# 6: (48 – 53) 
# 7: (60 – 100)

# FEMALE = 1
# 8: (0 – 2)
# 9: (4 – 6)
# 10: (8 – 12)
# 11: (15 – 20)
# 12: (25 – 32)
# 13: (38 – 43) 
# 14: (48 – 53) 
# 15: (60 – 100)
    '''
labels = {
    0: 'M,(0 to 2)',
    1: 'M,(4 to 6)',
    2: 'M,(8 to 12)',
    3: 'M,(15 to 20)',
    4: 'M,(25 to 32)',
    5: 'M,(38 to 43)',
    6: 'M,(48 to 53)',
    7: 'M,(60 to 100)',
    8: 'F,(0 to 2)',
    9: 'F,(4 to 6)',
    10: 'F,(8 to 12)',
    11: 'F,(15 to 20)',
    12: 'F,(25 to 32)',
    13: 'F,(38 to 43)', 
    14: 'F,(48 to 53)',
    15: 'F,(60 to 100)'
        }
colours = {0:(255,0,0),1:(0,255,0),2:(0,0,255)}

In [17]:
size = 48
src = cv2.VideoCapture(0)
while(True):
    ret,img = src.read()
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img_gray,1.3,5)
    
    for x,y,width,height in faces:
        
        # Extract face
        face = img_gray[y:y+height+20,x:x+width]
        
        # Reshape image
        face = cv2.resize(face,(size,size))
        face = np.reshape(face,(1, 1, size, size))           
  
        # Normalize
        face = face/255.0
        face = torch.Tensor(face)
 
        face = (face - mean)/std
        scores = model(face)
        _, face_pred = torch.max(scores.data, 1)
                
        string_output = labels[face_pred.item()]
        # Draw coloured rectangle to indicate face detection & label
        cv2.rectangle(img,(x,y),(x+width,y+height),colours[0],2)
        cv2.rectangle(img,(x,y-40),(x+width,y),colours[0],-1)
        cv2.putText(img,string_output,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',img)
    exit = cv2.waitKey(1)
    if (exit==27): # 27 == ACSII char for 'ESC'
        break
        
cv2.destroyAllWindows()
src.release()